In [25]:
poligon1 = [[0,1,0], [1,1,1], [0,1,0]]
poligon2 = [[1,1], [1,0]]
poligon3 = [[1,1,1], [0,1,0]]

poligons = [poligon1, poligon2, poligon3, poligon1, poligon2, poligon3, poligon1, poligon2, poligon3, poligon1, poligon2, poligon3]

matrix = [[0,0,0,0,0,0,0,0],
          [0,0,0,0,0,0,0,0],
          [0,0,0,0,0,0,0,0],
          [0,0,0,0,0,0,0,0],
          [0,0,0,0,0,0,0,0],
          [0,0,0,0,0,0,0,0],
          [0,0,0,0,0,0,0,0]]
pallets = []

In [26]:
def print_matrix(matrix):
    for i in range(len(matrix)):
        print(matrix[i],'\n')

# print_matrix(matrix)


In [27]:
def fit_item(matrix, item):
    for i in range(len(matrix[0])):
        for j in range(len(matrix)):
            exit = False
            # условия, что объект влезает
            if len(item) + i <= len(matrix) and len(item[0]) + j <= len(matrix[0]):
                # располагаем объект
                for p in range(len(item)):
                    for k in range(len(item[0])):
                        # решаем есть ли пересечение
                        if matrix[i+p][j+k] + item[p][k] == 2:
                            exit = True
                            break
                    if exit:
                        break
            else:
                exit = True
            # если пересечений нет и элемент влезает, то добавляем его
            if not exit:
                for p in range(len(item)):
                    for k in range(len(item[0])):
                        matrix[i+p][j+k] += item[p][k]
                break
        if not exit:
            break
    print_matrix(matrix)
    print('--------------')
    return matrix, exit

In [28]:
# fit_item(matrix, poligon2)

In [ ]:
def fit_list(matrix, items):
    for item in items:
        matrix, exit = fit_item(matrix, item)
        # print_matrix(matrix)
        # print('--------------')
    return matrix

In [29]:
# fit_list(matrix, poligons)

In [30]:
pallets.append(matrix)
for poligon in poligons:
    for pallet in pallets:
        pallet, exit = fit_item(pallet, poligon)
        print(pallets[-1])
        if exit and pallet == pallets[-1]:
            pallets.append(matrix)
        if not exit:
            break
    # if not exit:
    #         break



[0, 1, 0, 0, 0, 0, 0, 0] 

[1, 1, 1, 0, 0, 0, 0, 0] 

[0, 1, 0, 0, 0, 0, 0, 0] 

[0, 0, 0, 0, 0, 0, 0, 0] 

[0, 0, 0, 0, 0, 0, 0, 0] 

[0, 0, 0, 0, 0, 0, 0, 0] 

[0, 0, 0, 0, 0, 0, 0, 0] 

--------------
[[0, 1, 0, 0, 0, 0, 0, 0], [1, 1, 1, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0]]
[0, 1, 0, 1, 1, 0, 0, 0] 

[1, 1, 1, 1, 0, 0, 0, 0] 

[0, 1, 0, 0, 0, 0, 0, 0] 

[0, 0, 0, 0, 0, 0, 0, 0] 

[0, 0, 0, 0, 0, 0, 0, 0] 

[0, 0, 0, 0, 0, 0, 0, 0] 

[0, 0, 0, 0, 0, 0, 0, 0] 

--------------
[[0, 1, 0, 1, 1, 0, 0, 0], [1, 1, 1, 1, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0]]
[0, 1, 0, 1, 1, 1, 1, 1] 

[1, 1, 1, 1, 0, 0, 1, 0] 

[0, 1, 0, 0, 0, 0, 0, 0] 

[0, 0, 0, 0, 0, 0, 0, 0] 

[0, 0, 0, 0, 0, 0, 0, 0] 

[0, 0, 0, 0, 0, 0, 0, 0] 

[0, 0, 0, 0, 0, 0, 0, 0] 

--------------
[[0, 1, 0, 1, 1, 1, 1,

KeyboardInterrupt: 